 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8889/notebooks/notes/BOOKS/Programming%20Collective%20Intelligence/Chapter_3%20Discoving%20Groups/3.1%20Discovering%20Groups.ipynb#Hirearchial-Clustering" data-toc-modified-id="Hirearchial-Clustering-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Hirearchial Clustering</a></span><ul class="toc-item"><li><span><a href="http://localhost:8889/notebooks/notes/BOOKS/Programming%20Collective%20Intelligence/Chapter_3%20Discoving%20Groups/3.1%20Discovering%20Groups.ipynb#分级" data-toc-modified-id="分级-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>分级</a></span></li><li><span><a href="http://localhost:8889/notebooks/notes/BOOKS/Programming%20Collective%20Intelligence/Chapter_3%20Discoving%20Groups/3.1%20Discovering%20Groups.ipynb#绘制树状图" data-toc-modified-id="绘制树状图-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>绘制树状图</a></span></li><li><span><a href="http://localhost:8889/notebooks/notes/BOOKS/Programming%20Collective%20Intelligence/Chapter_3%20Discoving%20Groups/3.1%20Discovering%20Groups.ipynb#列聚类" data-toc-modified-id="列聚类-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>列聚类</a></span></li></ul></li><li><span><a href="http://localhost:8889/notebooks/notes/BOOKS/Programming%20Collective%20Intelligence/Chapter_3%20Discoving%20Groups/3.1%20Discovering%20Groups.ipynb#K-Means-Clustering" data-toc-modified-id="K-Means-Clustering-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>K-Means Clustering</a></span></li><li><span><a href="http://localhost:8889/notebooks/notes/BOOKS/Programming%20Collective%20Intelligence/Chapter_3%20Discoving%20Groups/3.1%20Discovering%20Groups.ipynb#Clusters-of-Prefernce" data-toc-modified-id="Clusters-of-Prefernce-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Clusters of Prefernce</a></span></li><li><span><a href="http://localhost:8889/notebooks/notes/BOOKS/Programming%20Collective%20Intelligence/Chapter_3%20Discoving%20Groups/3.1%20Discovering%20Groups.ipynb#Viewing-Data-in-2D" data-toc-modified-id="Viewing-Data-in-2D-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Viewing Data in 2D</a></span></li></ul></div>

<font size=6.5 color='blue'>Discovering Groups</font>

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import pandas as pd
import numpy as np

In [4]:
blogdata=pd.read_table('blogdata.txt')

# Hirearchial Clustering
分级聚类

## 分级
- 定义一个类，缓存各行必要的信息
- 嵌套结构，来进行聚类
- 递归搜索，打印结果

读入数据

In [94]:
def readfile(filename):
    file = open(filename, 'r')
    lines = [line for line in file]

    colnames = lines[0].strip().split('\t')[1:]
    rownames = []
    data = []
    for line in lines[1:]:
        p = line.strip().split('\t')
        rownames.append(p[0])
        data.append([float(x) for x in p[1:]])
    return rownames, colnames, data


blognames, words, data = readfile('blogdata.txt')

**紧密度计算**
- `1-pearsonr`

In [61]:
from math import sqrt
def pearson(v1,v2):
    sum1=sum(v1)
    sum2=sum(v2)

    # Sums of the squares
    sum1Sq=sum([pow(v,2) for v in v1])
    sum2Sq=sum([pow(v,2) for v in v2])	

    # Sum of the products
    pSum=sum([v1[i]*v2[i] for i in range(len(v1))])

    # Calculate r (Pearson score)
    num=pSum-(sum1*sum2/len(v1))
    den=sqrt((sum1Sq-pow(sum1,2)/len(v1))*(sum2Sq-pow(sum2,2)/len(v1)))
    if den==0: return 0

    return 1.0-num/den

# from scipy.stats import pearsonr
# def pearson(v1,v2):
#     r,p_value = pearsonr(v1,v2)
#     if p_value==1: r=0
#     return 1.0 - r

In [107]:
class bicluster:
    def __init__(self,vec,left=None,right=None,distance=0.0, i_d=None):
        self.left=left
        self.right=right
        self.vec=vec
        self.i_d=i_d
        self.distance = distance

In [108]:
def hcluster(rows, distance=pearson):
    distances = {}
    currentclustid = -1

    clust = [bicluster(rows[i], i_d=i) for i in range(len(rows))]

    while len(clust) > 1:
        lowestpair = (0, 1)
        closest = distance(clust[0].vec, clust[1].vec)

        for i in range(len(clust)):
            for j in range(i + 1, len(clust)):
                if (clust[i].i_d, clust[j].i_d) not in distances:
                    distances[(clust[i].i_d, clust[j].i_d)] = distance(
                        clust[i].vec, clust[j].vec)
                d = distances[(clust[i].i_d, clust[j].i_d)]

                if d < closest:
                    closest = d
                    lowestpair = (i, j)
        mergevec = [
            (clust[lowestpair[0]].vec[i] + clust[lowestpair[1]].vec[i]) / 2.0
            for i in range(len(clust[0].vec))
        ]

        newcluster = bicluster(
            mergevec,
            left=clust[lowestpair[0]],
            right=clust[lowestpair[1]],
            distance=closest,
            i_d=currentclustid)
        #构建立新的 bicluster 对象，并将上级两个 bicluster 元素写入 left/right属性

        currentclustid -= 1
        del clust[lowestpair[1]]  #先删除大序号，再删除小序号，否则序号会乱
        del clust[lowestpair[0]]
        clust.append(newcluster)  #i_d为负数，且放在clust最后

    return clust[0], distances

In [64]:
top_clust,distances = hcluster(data)

In [65]:
distances = [(distance, pair) for (pair, distance) in distances.items()]
distances.sort()

In [66]:
def printdis(clust):
    if clust.distance !=0:
        print(clust.distance) 
    if clust.left!=None: printdis(clust.left)
    if clust.right!=None: printdis(clust.right)
        
# printdis(top_clust)

In [67]:
def printclust(clust, labels=None, n=0):
    for i in range(n):
        print(' ', end='')
    if clust.i_d < 0:
        print('-')
    else:
        if labels == None: print(clust.i_d)
        else: print(labels[clust.i_d])

    #递归搜索
    if clust.left != None: printclust(clust.left, labels=labels, n=n + 1)
    if clust.right != None: printclust(clust.right, labels=labels, n=n + 1)

In [68]:
# printclust(top_clust, labels=blognames)

## 绘制树状图

In [69]:
def gethight(clust):
    if clust.left==None and clust.right==None: return 1
    
    return gethight(clust.left) + gethight(clust.right)

def getdepth(clust):
    if clust.left==None and clust.right==None: return 0
    
    return max(getdepth(clust.left), getdepth(clust.right)) + clust.distance

In [70]:
from PIL import Image, ImageDraw

In [16]:
def drawdendrogram(clust, labels, jpeg='clusters.jpg'):
    h = gethight(clust)*20
    w = 1200
    depth = getdepth(clust)
    
    scaling = float(w-150)/depth     #w-150：留出两边的间隙；可用宽度/depth=放大系数
    
    img= Image.new('RGB',(w,h),(255,255,255))
    draw= ImageDraw.Draw(img)
    
    draw.line((0,h/2,10,h/2),fill=(255,0,0))
    
    drawnode(draw,clust,10,(h/2),scaling,labels)
    img.save(jpeg,'JPEG')

def drawnode(draw,clust,x,y,scaling,labels):
    if clust.i_d<0:
        h1=gethight(clust.left) *20
        h2 = gethight(clust.right) * 20
        top = y-(h1+h2)/2        #坐标？？
        bottom = y+ (h2+h1)/2    #坐标？？
        
        l1=clust.distance * scaling
        
        draw.line((x,top+h1/2,x,bottom-h2/2),fill=(255,0,0))
        
        draw.line((x,top+h1/2,x+l1,top+h1/2),fill=(255,0,0))          #坐标？？
        draw.line((x,bottom-h2/2,x+l1,bottom-h2/2),fill=(255,0,0))    #坐标？？
        
        drawnode(draw,clust.left,x+l1,top+h1/2,scaling,labels)
        drawnode(draw,clust.right,x+l1,bottom-h2/2,scaling,labels)
        
    else:
        draw.text((x+5,y-7),labels[clust.i_d],(0,0,0))

In [17]:
drawdendrogram(top_clust,blognames,jpeg="blogclust.jpg")

In [18]:
img =Image.new('RGB',(128,128),(250,244,234))
draw= ImageDraw.Draw(img)
draw.line((0,0,30,60),fill=(255,0,0))
draw.text((0,0),'(0,0)',(0,0,0,))
draw.text((30,60),'(x=30,y=60)',(0,0,0,))
img.save('zeropoint.jpg','JPEG')

**`Image`对象原点**
- 位置：左上角
- `y`：向下增大
- `x`：向右增大![zeropoint](zeropoint.jpg)

## 列聚类
将原来的数据进行转置后即可利用原来的函数对列进行聚类

In [19]:
def rotatematrix(data):
    rdata=[]
    row=[]
    for j in range(len(data[0])):
        row=[data[i][j] for i in range(len(data))]
        rdata.append(row)
    return rdata

In [20]:
rdata=rotatematrix(data)

In [21]:
wordclust,word_distances=hcluster(rdata)

In [22]:
drawdendrogram(wordclust,labels=words,jpeg='wordclust.jpg')

<font size=2, color='red'>**稀疏数据**</font><br>
shape of rdata：{{(len(rdata[0]),len(rdata))}}
<br>待聚类项目（单词）的数量({{len(rdata)}}) > 数据的数量({{len(rdata[0])}})，出现无意义聚类的可能性就增加

# K-Means Clustering
分组聚类

In [23]:
import random

In [88]:
def kcluster(rows, distance=pearson, k=4, isprint=False):
    #确定每一维度的最小值/最大值
    ranges = [(min([row[i] for row in rows]), max([row[i] for row in rows]))
              for i in range(len(rows[0]))]

    clusters = [[random.random() * (ranges[i][1] - ranges[i][0] + ranges[i][0]) for i in range(len(ranges))] 
                for j in range(k)]
    
    lastmatches = None
    for t in range(100): #最多迭代100次
        if isprint: print('Iterarion %d' %t)
        bestmatches = [[] for i in range(k)]
        
        for j in range(len(rows)):
            row = rows[j]
            bestmatch=0         #与cluster中的最近的点的index
            
            for i in range(k):
                d = distance(clusters[i], row)
                if d < distance(clusters[bestmatch], row): bestmatch=i
            bestmatches[bestmatch].append(j)
        
        if bestmatches==lastmatches:break
        lastmatches = bestmatches
        
        #更新clusters位置
        for i in range(k):
            avgs = [0.0] *len(rows[0])    #[1] * n：得到一个有n个1的列表，
            for rowid in bestmatches[i]:
                for m in range(len(rows[rowid])):
                    avgs[m] += rows[rowid][m]
            for j in range(len(avgs)):
                if len(bestmatches[i])>0:
                    avgs[j] /=len(bestmatches[i])

            clusters[i]=avgs
    return bestmatches

In [89]:
kcluster(data,k=6,isprint=True)

Iterarion 0
Iterarion 1
Iterarion 2
Iterarion 3
Iterarion 4
Iterarion 5


[[0,
  1,
  3,
  6,
  9,
  10,
  11,
  14,
  15,
  18,
  19,
  20,
  22,
  26,
  28,
  29,
  31,
  33,
  35,
  37,
  38,
  39,
  43,
  47,
  51,
  57,
  58,
  61,
  64,
  70,
  71,
  72,
  73,
  74,
  76,
  77,
  78,
  82,
  86,
  87,
  88,
  89,
  92,
  93,
  95,
  96,
  97,
  98],
 [2,
  7,
  13,
  16,
  21,
  23,
  24,
  34,
  36,
  40,
  49,
  56,
  62,
  69,
  79,
  80,
  84,
  85,
  91,
  94],
 [4, 27, 41, 42, 45, 46, 81],
 [44, 55, 83],
 [8, 12, 25, 30, 32, 50, 52, 53, 54, 59, 63, 65, 67, 90],
 [5, 17, 48, 60, 66, 68, 75]]

# Clusters of Prefernce
针对偏好的聚类
<font size=3> **距离度量标准**</font><br>
**Tanimoto系数**
- 交集占并集的比例
- 数据集数据值为0/1：无/有

In [124]:
def tanimoto(v1,v2):
    c1,c2,shr=0,0,0
    
    for i in range(len(v1)):
        if v1[i] !=0: c1+=1
        if v2[i] !=0: c2+=1
        if v1[i] !=0 and v2[i] !=0: shr+=1
        
    return 1.0 - (float(shr)/(c1+c2-shr+0.00001))   #+0.00001，防止除以0报错，对人进行聚类时：两个爱好一样的人返回距离约等于0

In [115]:
wants, people, data = readfile('zebo.txt')
wants_clust, distances = hcluster(data,distance = tanimoto)
drawdendrogram(wants_clust,wants,jpeg='wants_cluster.jpg')

In [122]:
rdata=rotatematrix(data)
people_clust, distances = hcluster(rdata,distance = tanimoto)
drawdendrogram(people_clust,people,jpeg='people_cluster.jpg')

# Viewing Data in 2D 